In [ ]:
import os
import numpy as np
import requests
from PIL import Image
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical


In [ ]:
emoji_classes = [
    "😀", "😂", "😢", "😭", "😡", "🤬", "😱", "😮", "😎", "🤓",
    "🤠", "😇", "😈", "🤖", "🤯", "🤢", "🤮", "🥵", "🥶", "😴"
]

num_classes = len(emoji_classes)
img_size = 64


In [ ]:
def emoji_to_codepoint(e):
    return "-".join(f"{ord(c):x}" for c in e)


In [ ]:
TWEMOJI_BASE = "https://cdnjs.cloudflare.com/ajax/libs/twemoji/14.0.2/72x72/"
SAVE_DIR = "twemoji_png"
os.makedirs(SAVE_DIR, exist_ok=True)

def download_emoji_png(emoji):
    code = emoji_to_codepoint(emoji)
    url = f"{TWEMOJI_BASE}{code}.png"
    save_path = os.path.join(SAVE_DIR, f"{code}.png")

    if not os.path.exists(save_path):
        print("Downloading:", emoji, url)
        r = requests.get(url)
        with open(save_path, "wb") as f:
            f.write(r.content)
    
    return save_path

emoji_png_paths = {emoji: download_emoji_png(emoji) for emoji in emoji_classes}


In [ ]:
def generate_color_emoji_image(emoji):
    bg_color = tuple(np.random.randint(0,255,size=3))
    img = Image.new("RGB", (img_size, img_size), bg_color)

    emoji_img = Image.open(emoji_png_paths[emoji]).convert("RGBA")
    emoji_img = emoji_img.resize((50, 50), Image.LANCZOS)

    pos = ((img_size - emoji_img.width)//2,
           (img_size - emoji_img.height)//2)

    img.paste(emoji_img, pos, emoji_img)

    return np.array(img)


In [ ]:
images_per_class = 200
dataset_images = []
dataset_labels = []

print("Generating dataset...")

for label, emoji in enumerate(emoji_classes):
    print(f"Class {label}: {emoji}")
    for _ in range(images_per_class):
        img = generate_color_emoji_image(emoji)
        dataset_images.append(img)
        dataset_labels.append(label)

dataset_images = np.array(dataset_images)
dataset_labels = np.array(dataset_labels)

print("Dataset complete!")
print("Images:", dataset_images.shape)
print("Labels:", dataset_labels.shape)


In [ ]:
plt.figure(figsize=(12,4))
for i in range(10):
    idx = random.randint(0, len(dataset_images)-1)
    plt.subplot(2,5,i+1)
    plt.imshow(dataset_images[idx])
    plt.axis("off")
    plt.title(emoji_classes[dataset_labels[idx]])
plt.tight_layout()
plt.show()


In [ ]:
images = dataset_images.astype("float32") / 255.0
labels_cat = to_categorical(dataset_labels, num_classes=num_classes)

X_train, X_test, y_train, y_test = train_test_split(
    images, labels_cat, test_size=0.2,
    stratify=dataset_labels,
    random_state=42
)

X_train.shape, X_test.shape


In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


In [ ]:
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=32
)


In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history["accuracy"], label="Train Acc")
plt.plot(history.history["val_accuracy"], label="Val Acc")
plt.legend()
plt.title("Accuracy")

plt.subplot(1,2,2)
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.legend()
plt.title("Loss")

plt.show()


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_acc)


In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(12,10))
sns.heatmap(cm, annot=True, fmt="d",
            xticklabels=emoji_classes,
            yticklabels=emoji_classes,
            cmap="Blues")
plt.title("Confusion Matrix")
plt.show()
